<a href="https://colab.research.google.com/github/r3lativo/fine-tuning-for-sa/blob/main/lama3_8b_ft_sa_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU bitsandbytes datasets accelerate transformers peft huggingface_hub evaluate trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import userdata
hugginface_token = userdata.get('HF_TOKEN')
!huggingface-cli login --token $hugginface_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import torch
from transformers import (  # Huggingface transformers
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import numpy
import transformers
from datasets import load_dataset
from trl import SFTTrainer
import numpy as np
import evaluate


In [4]:
accuracy = evaluate.load("accuracy")

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
# LLAMA3 pre-training doesn't have EOS token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [8]:
# Define label maps, to be able to pass from labels to id and vice versa
id2label = {0: "NEG", 1: "POS"}
label2id = {"NEG":0, "POS":1}


#### Model Architecture

It's important to observe the model's construction so you can ensure you know which modules you should apply LoRA to.

As per the paper, we're going to focus on the attention weights - so keep an eye out for modules like: `q_proj`, `v_proj`, `query_key_value`.

In [9]:
# specify how to quantize the model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load the model on our GPU in a quantized state
    bnb_4bit_use_double_quant=True,  # Quantize quantized weights
    bnb_4bit_quant_type="nf4",  # information "theoretically" optimal dtype for normally distributed weights

    # Storing numbers in 4-bit is great, working with them is instead very bad!
    # We DEQUANTIZE them each time we operate with them and requantize later.
    # This dequantize-quantize operation will impact a bit the inference time.
    # It is a trade-off we have to pay.
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],# "v_proj", "k_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    id2label=id2label,
    label2id=label2id,
    num_labels=2,
    device_map="auto"
)
model

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

In [12]:
model.config.pad_token_id = model.config.eos_token_id

In [13]:
model = prepare_model_for_kbit_training(model)
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

In [14]:
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
             

In [ ]:
model.print_trainable_parameters()

In [15]:
dataset = load_dataset("stanfordnlp/imdb")
print(dataset)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [16]:
# Create a smaller version of the dataset, to speed up the training
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(100))
small_eval_dataset = dataset['test'].shuffle(seed=42).select(range(100))

In [17]:
print(small_train_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}


In [18]:
def tokenize_function(examples):
    """Helper function to tokenize text from the Dataset"""
    text = examples["text"]

    # Tokenize and truncate
    tokenizer.truncation_side = 'left'
    tokenized_inputs = tokenizer(
        text,
        max_length=512,
        truncation=True,
        padding=True,
        return_tensors="np",
        )

    return tokenized_inputs


In [19]:
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [20]:
print(small_train_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1, 'input_ids': [128000, 3947, 374, 912, 12976, 520, 682, 1990, 11246, 1291, 323, 8626, 5888, 719, 279, 2144, 430, 2225, 527, 4379, 4101, 922, 16806, 17073, 13, 8626, 5888, 5992, 73624, 11, 11246, 1291, 5992, 11670, 13

In [21]:
# Check the Positive to total ratio for the shuffled smaller datasets
a = sum(small_train_dataset['label'])/len(small_train_dataset['label'])
b = sum(small_eval_dataset['label'])/len(small_eval_dataset['label'])

print(f"Positive to total ratio in train: {a*100}%")
print(f"Positive to total ratio in eval:  {b*100}%")

Positive to total ratio in train: 47.0%
Positive to total ratio in eval:  47.0%


In [22]:
# Create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
def test_model(model, data_slice):
    """Helper function to test the model on small set of sentences from data"""

    print("Model predictions:")
    print(f"|Text{' '*50}|Gold |Pred |")
    print(f"{'-'*50}")

    correct_count = 0

    for e in data_slice:
        text = e['text']
        label = id2label[e['label']]

        # Tokenize text
        inputs = tokenizer.encode(
            text,
            max_length=512,
            truncation=True,
            padding=True,
            return_tensors="pt"
            ).to("cuda")

        # Compute logits
        with torch.no_grad(): logits = model(inputs).logits
        # Convert logits to label
        predictions = torch.argmax(logits)

        l_pred = id2label[predictions.tolist()]
        print(f"|{text[:50]}... |{label}  |{l_pred}  |")

        # Keep track of correct prediction for accuracy
        if label == l_pred: correct_count += 1

    print(f"{'-'*50}")
    print(f"Accuracy: {(correct_count/10)*100}%")


In [24]:
# Test model on ten random sentences
example_data = small_eval_dataset.shuffle().select(range(10))
test_model(model, example_data)

Model predictions:
|Text                                                  |Gold |Pred |
--------------------------------------------------
|This movie is really bad, trying to create scienti... |NEG  |POS  |
|Late night on BBC1, was on my way to bed but curio... |POS  |POS  |
|Intended as light entertainment, this film is inde... |NEG  |POS  |
|The story is: a turn-of-the-century troupe of acto... |NEG  |NEG  |
|All I can do is echo the sentiment already express... |NEG  |NEG  |
|I saw Le Conseguenze Dell'Amore on the 2005 Rotter... |POS  |POS  |
|Film critics of the world, I apologize. It is your... |NEG  |POS  |
|Well, where to start describing this celluloid deb... |NEG  |POS  |
|this movie makes me laugh by even just thinking ab... |POS  |POS  |
|This is the latest entry in the long series of fil... |POS  |POS  |
--------------------------------------------------
Accuracy: 60.0%


In [25]:
def compute_metrics(p):
    """Helper function to evaluate the trainer"""
    #print(p)
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    return {"accuracy": accuracy.compute(predictions=pred, references=labels)}


In [34]:
# Training hyperparameters
lr = 1e-4
batch_size = 8
num_epochs = 1
optimizer = 'paged_adamw_8bit' # from the QLoRA paper

In [35]:
training_args = TrainingArguments(
    output_dir="llama3-8b-ft-sa",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    optim=optimizer,
    learning_rate=lr,
    logging_steps=1,
    bf16=True, # ensure proper upcasting for compute dtypes
    weight_decay = 0.01,
    eval_strategy = 'epoch',
    #save_strategy = 'epoch',
    #load_best_model_at_end = True
)

In [36]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    #max_seq_length=max_seq_length,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [37]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Accuracy
1,0.420400,1.023601,{'accuracy': 0.52}


Trainer is attempting to log a value of "{'accuracy': 0.52}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=13, training_loss=1.0080350270638099, metrics={'train_runtime': 1308.3487, 'train_samples_per_second': 0.076, 'train_steps_per_second': 0.01, 'total_flos': 2144778741350400.0, 'train_loss': 1.0080350270638099, 'epoch': 1.0})

In [38]:
# Empirical test after fine-tuning
test_model(model, example_data)

Model predictions:
|Text                                                  |Gold |Pred |
--------------------------------------------------
|This movie is really bad, trying to create scienti... |NEG  |NEG  |
|Late night on BBC1, was on my way to bed but curio... |POS  |NEG  |
|Intended as light entertainment, this film is inde... |NEG  |POS  |
|The story is: a turn-of-the-century troupe of acto... |NEG  |NEG  |
|All I can do is echo the sentiment already express... |NEG  |NEG  |
|I saw Le Conseguenze Dell'Amore on the 2005 Rotter... |POS  |NEG  |
|Film critics of the world, I apologize. It is your... |NEG  |NEG  |
|Well, where to start describing this celluloid deb... |NEG  |POS  |
|this movie makes me laugh by even just thinking ab... |POS  |NEG  |
|This is the latest entry in the long series of fil... |POS  |POS  |
--------------------------------------------------
Accuracy: 50.0%
